目标检测的边框损失函数
===
IOU(Intersection Over Union)在目标检测中是一个非常重要的指标，NMS(Non-Max Supression非极大值抑制)是过滤重复预测框的非常重要的一个办法

# 1.Smooth L1损失函数

## 1.1.Smooth L1损失函数介绍

假设x为预测框和真实框之间的数值差异，常用的L1和L2 Loss定义为
$$
\begin{cases}
L_1=\left| x \right| \frac{dL_2(x)}{x}=2x \\
L_2=x^2 \\
Smooth_{L_1}=\begin{cases}
0.5x^2 & \left| x \right| < 1 \\
\left| x \right| -0.5 & otherwise
\end{cases}
\end{cases}
$$

## 1.2.损失函数的导数

$$
\frac{dL_1(x)}{x}=
\begin{cases}
1 & x \geq 0 \\
-1 & otherwise
\end{cases}
$$

$$\frac{dL_2(x)}{x}=2x$$

$$
\frac{dSmooth_{L_1}(x)}{x}=
\begin{cases}
x & \left| x \right| < 1 \\
\pm{1} & otherwise
\end{cases}
$$

## 1.3. 损失函数总结
从损失函数对x的导数可知： $L_1$损失函数对x的导数为常数，在训练后期，x很小时，如果learning rate 不变，损失函数会在稳定值附近波动，很难收敛到更高的精度。 $L_2$损失函数对x的导数在x值很大时，其导数也非常大，在训练初期不稳定。$Smooth_{L_1}$完美的避开了 $L_1$和$L_2$损失的缺点。

## 1.4. 实际目标检测框回归任务中的损失loss

$$L_{loc}(t^u, v)=\sum_{i \in \{x,y,w,h\}}Smooth_{L_1}(t_i^u-v_i)$$

其中$v=(v_x,v_y,v_w,v_h)$表示GT的框坐标，$t^u=(t_x^u,t_y^u,t_w^u,t_h^u)$表示预测的框坐标，即分别求4个点的loss，然后相加作为Bounding Box Regression Loss。

![Images](images/03_01_000.jpg)

## 1.5.$Smooth_{L_1}$的缺点
- 上面的三种Loss用于计算目标检测的Bounding Box Loss时，独立的求出4个点的Loss，然后进行相加得到最终的Bounding Box Loss，这种做法的假设是4个点是相互独立的，实际是有一定相关性的
- 实际评价框检测的指标是使用IOU，这两者是不等价的，多个检测框可能有相同大小的$Smooth_{L_1}(x)$ Loss，但IOU可能差异很大，为了解决这个问题就引入了IOU LOSS。

# 2.IOU

## 2.1.通过4个坐标点独立回归Building boxes的缺点

- 检测评价的方式是使用IoU,而实际回归坐标框的时候是使用4个坐标点，如下图所示，是不等价的；L1或者L2 Loss相同的框，其IoU 不是唯一的
- 通过4个点回归坐标框的方式是假设4个坐标点是相互独立的，没有考虑其相关性，实际4个坐标点具有一定的相关性
- 基于L1和L2的距离的loss对于尺度不具有不变性

## 2.2.IOU所引起的问题
![image](Images/03_01_003.jpg)

图(a)中的三组框具有相同的L2 Loss，但其IoU差异很大；图(b)中的三组框具有相同的L1 Loss,但IoU 同样差异很大，说明L1,L2这些Loss用于回归任务时，不能等价于最后用于评测检测的IoU.

IoU 是目标检测领域最重要的评价尺度之一，特性是对尺度不敏感，主要判断检测框的重合程度。但是对于CNN而言，没有方向信息，无法反馈神经网络的边界回归框应该如何调整。IoU作为度量和损失都有两个主要问题
- 如果两个对象不重叠，IoU值将为零，并且不能反映两个形状之间的距离。在非重叠对象的情况下，如果使用IoU作为损失，则梯度为零，无法优化
- IoU不能正确区分两个对象的不同对齐。更精确地说，对于两个具有相同交点水平的不同方向重叠的对象，IoU将是完全相等，因此，IoU函数的值并不能反映两个对象是如何重叠的。

![image](Images/03_01_004.jpg)

于是通过将IoU的概念扩展到非重叠的情况来解决IoU的这两个弱点。这个推广遵循三个条件
- 与IOU相同的定义，即将比较对象的形状属性编码到区域属性中
- 保持IOU的尺度不变性质
- 确保在发生重叠物体时与IOU有很强的相关性

## 2.3.IOU介绍
在目标检测的评价体系中，有一个参数叫做IoU，简单来讲就是模型产生的目标窗口和原来标记窗口的交叠率。具体我们可以简单的理解为检测结果(DetectionResult)与Ground Truth的交集比上它们的并集，即为检测的准确率IoU
$$IOU=\frac{DR \cap GT}{DR \cup GT}$$
![image](Images/03_01_001.png)

GT = GroundTruth; DR = DetectionResult; 黄色边框框起来的是$GT \cap DR$，绿色框框起来的是$GT \cup DR$,当然最理想的情况就是 DR 与 GT 完全重合，即IOU=1

![image](Images/03_01_002.png)

基于此提出IoU Loss,其将4个点构成的box看成一个整体进行回归

![Images](images/03_01_008.jpg)

上图中的红色点表示目标检测网络结构中Head部分上的点（i,j），绿色的框表示Ground truth框, 蓝色的框表示Prediction的框，IoU loss的定义如上，先求出2个框的IoU，然后再求个-ln(IoU)，实际很多是直接定义为IoU Loss = 1-IoU

# 2.GIOU(Generalized IOU)-目标检测任务的新Loss
对于两个任意凸型(体积)$A,B \subseteq S \in R^n$，首先求出了包含A和B的最小凸型$C \subseteq \in R^n$，为了比较两种特定的几何形状，C可以来自同一类型

我们找到一个最小的封闭形状C，让C可以把A，B包含在内，然后我们计算C中没有覆盖A和B的面积占C总面积的比值，然后用A与B的IoU减去这个比值

![image](Images/03_01_005.jpg)

## 2.1.GIOU的性质
1. 与IOU类似，GIOU也可以作为一个距离，loss可以用下面的公式计算$L_{GIOU}=1-L_{GIOU}$
2. 同原始IoU类似，GIoU对物体的大小不敏感。GIoU总是小于等于IoU，对于IoU，有:$0 \leq IOU \leq 1$，而GIOU则是$-1 \leq GIOU \leq 1$
3. 由于GIoU引入了包含A，B两个形状的C，所以当A，B不重合时，依然可以进行优化
4. 与IoU相比，GIoU不仅关注重叠区域。封闭形状C中两个对称形状A和B之间的空隙在A和B之间没有很好的对齐时增大,因此GIoU的值更能反映两个对称物体之间是如何重叠的

总之就是保留了IoU的原始性质同时弱化了它的缺点

## 2.2.GIOU作为BBox回归的损失
假设有输入为预测框$B^p=(x)1^p,y_1^p,x_2^p,y_2^p)$和真实框$B_g=(x_1^g,y_1^g,x_2^g,y_2^g)$。输出为$L_{IOU}$和$L_{GIOU}$
- 对于预测框$B^p$来说，有
$$x_2^p > x_1^p, y_2^p > y_1^p$$
且有
$$
\begin{split}
\hat{x}_1^p&=min(x_1^p,x_2^p) \\
\hat{x}_2^p&=max(x_1^p,x_2^p) \\
\hat{y}_1^p&=min(y_1^p,y_2^p) \\
\hat{y}_2^p&=max(y_1^p,y_2^p)
\end{split}
$$
- 计算$B^g$的面积
$$A^g=(x_2^g-x_1^g) \times (y_2^g-y_1^g)$$
- 计算$B^p$的面积
$$A^p=(x_2^p-x_1^p) \times (y_2^p-y_1^p)$$
- 计算$B^g,B^p$的重叠面积
假设如下
$$
\begin{split}
x_1^I&=max(\hat{x}_1^p,x_1^g) \\
x_2^I&=min(\hat{x}_2^p,x_2^g) \\
y_1^I&=max(\hat{y}_1^p,y_1^g) \\
y_2^I&=max(\hat{y}_2^p,y_2^g)
\end{split}
$$
有
$$
I=\begin{cases}
(x_2^I-x_1^I) \times (y_2^I-y_1^I) & x_2^I > x_1^I,y_2^i > y_1^I \\
0 & otherwise
\end{cases}
$$
- 找到可以包含$B^p,B^g$的最小的box $B^c$
$$
\begin{split}
x_1^c&=min(\hat{x}_1^p,x_1^g) \\
x_2^c&=max(\hat{x}_2^p,x_2^g) \\
y_1^c&=min(\hat{y}_1^p,y_1^g) \\
y_2^c&=max(\hat{y}_2^p,y_2^g)
\end{split}
$$
- 计算$B^c$的面积
$$A^c=(x_2^c-x_1^c) \times (y_2^c-y_1^c)$$
- 计算IOU
$$IOU=\frac{I}{U}=\frac{I}{A^p+A^g-I}$$
- 计算GIOU
$$GIOU=IOU-\frac{A^c-U}{A^c}$$
- 计算损失
$$L_{GIOU}=1-GIOU$$

## 2.3.计算GIOU

In [1]:
import numpy as np
def bbox_overlaps_giou(bboxes1, bboxes2):
    """Calculate the gious between each bbox of bboxes1 and bboxes2.
    Args:
        bboxes1(ndarray): shape (n, 4)
        bboxes2(ndarray): shape (k, 4)
    Returns:
        gious(ndarray): shape (n, k)
    """


    bboxes1 = bboxes1.astype(np.float32)
    bboxes2 = bboxes2.astype(np.float32)
    rows = bboxes1.shape[0]
    cols = bboxes2.shape[0]
    ious = np.zeros((rows, cols), dtype=np.float32)
    if rows * cols == 0:
        return ious
    exchange = False
    if bboxes1.shape[0] > bboxes2.shape[0]:
        bboxes1, bboxes2 = bboxes2, bboxes1
        ious = np.zeros((cols, rows), dtype=np.float32)
        exchange = True
    area1 = (bboxes1[:, 2] - bboxes1[:, 0] + 1) * (
        bboxes1[:, 3] - bboxes1[:, 1] + 1)
    area2 = (bboxes2[:, 2] - bboxes2[:, 0] + 1) * (
        bboxes2[:, 3] - bboxes2[:, 1] + 1)
    for i in range(bboxes1.shape[0]):
        x_start = np.maximum(bboxes1[i, 0], bboxes2[:, 0])
        x_min = np.minimum(bboxes1[i, 0], bboxes2[:, 0])
        y_start = np.maximum(bboxes1[i, 1], bboxes2[:, 1])
        y_min = np.minimum(bboxes1[i, 1], bboxes2[:, 1])
        x_end = np.minimum(bboxes1[i, 2], bboxes2[:, 2])
        x_max = np.maximum(bboxes1[i, 2], bboxes2[:, 2])
        y_end = np.minimum(bboxes1[i, 3], bboxes2[:, 3])
        y_max = np.maximum(bboxes1[i, 3], bboxes2[:, 3])

        overlap = np.maximum(x_end - x_start + 1, 0) * np.maximum(y_end - y_start + 1, 0)
        closure = np.maximum(x_max - x_min + 1, 0) * np.maximum(y_max - y_min + 1, 0)

        union = area1[i] + area2 - overlap
        closure
        ious[i, :] = overlap / union - (closure - union) / closure
    if exchange:
        ious = ious.T
    return ious

In [2]:
box1 = np.array([[0,0,100,100]]*3)
box2 = np.array([[0,200,100,300]]*3)
print(bbox_overlaps_giou(box1, box2))

[[-0.32890365 -0.32890365 -0.32890365]
 [-0.32890365 -0.32890365 -0.32890365]
 [-0.32890365 -0.32890365 -0.32890365]]


# 3.DIOU：更加稳定有效的目标框回归损失

## 3.1.IOU回顾
$$
\begin{cases}
IOU&=\frac{\left|B \cap B^{gt} \right|}{\left|B \cup B^{gt} \right|} \\
\mathcal{L}_{IOU}&=1-IOU=1-\frac{\left|B \cap B^{gt} \right|}{\left|B \cup B^{gt} \right|}
\end{cases}
$$

IoU的缺点是当两个框不想相交时，IoU损失总是1，不能给出优化方向。

## 3.2.GIOU回顾
$$
\begin{cases}
GIOU&=IOU-\frac{\left|C-B \cup B^{gt} \right|}{\left| C \right|} \\
\mathcal{L}_{GIOU}&=1-IOU+\frac{\left|C-B \cup B^{gt} \right|}{\left| C \right|}
\end{cases}
$$

GIOU在IOU的基础上添加了一个项，其中C表示包含两个框的最小矩形，因此可以优化两个框不相交的情况。不过，GIOU也存在着问题。当两个框相交时，GIOU损失退化为IOU损失。因此，当包含预测框bbox和地面真值bbox时，特别是在水平和垂直方向上，很难进行优化。这里的一个猜想是，在水平方向和垂直方向上，损失的值没有在其他方向上增长得快，因此对这两个方向的惩罚不够，导致收敛速度减慢

![Images](images/03_01_006.jpeg)

在上图的情况下，GIoU损耗退化为IoU损耗，而提出DIoU损耗仍然可以区分。 绿色和红色分别表示目标框和预测框

## 3.3.总结
- IOU：从IOU误差的曲线可以发现，anchor越靠近边缘，误差越大，那些与目标框没有重叠的anchor基本无法回归。
- GIOU：从GIoU误差的曲线我们可以发现，对于一些没有重叠的anchor，GIOU的表现要比IOU更好。但是由于GIOU仍然严重的依赖IOU，因此在两个垂直方向，误差很大，基本很难收敛，这就是GIOU不稳定的原因。

## 3.4.DIOU
$$
\begin{cases}
DIOU&=IOU-\frac{\rho(b,b^{gt})}{c^2} \\
\mathcal{L}_{DIOU}&=1-IOU+\frac{\rho(b,b^{gt})}{c^2}
\end{cases}
$$

$b，b^{gt}$分别代表了anchor框和目标框的中心点，且$\rho$代表的是计算两个中心点间的欧式距离。$c$代表的是能够同时覆盖anchor和目标框的最小矩形的对角线距离。因此DIoU中对anchor框和目标框之间的归一化距离进行了建模。直观的展示如下图所示。

![Images](images/03_01_007.jpeg)

## 3.5.DIOU的优点
- 与GIOU loss类似，DIOU loss在与目标框不重叠时，仍然可以为边界框提供移动方向。
- DIOU loss可以直接最小化两个目标框的距离，因此比GIOU loss收敛快得多。
- 对于包含两个框在水平方向和垂直方向上这种情况，DIOU损失可以使回归非常快，而GIOU损失几乎退化为IOU损失。

In [ ]:
import torch

def Diou(bboxes1, bboxes2):
    rows = bboxes1.shape[0]
    cols = bboxes2.shape[0]
    dious = torch.zeros((rows, cols))
    if rows * cols == 0:
        return dious
        
    exchange = False
    if bboxes1.shape[0] > bboxes2.shape[0]:
        bboxes1, bboxes2 = bboxes2, bboxes1
        dious = torch.zeros((cols, rows))
        exchange = True

    w1 = bboxes1[:, 2] - bboxes1[:, 0]
    h1 = bboxes1[:, 3] - bboxes1[:, 1]
    w2 = bboxes2[:, 2] - bboxes2[:, 0]
    h2 = bboxes2[:, 3] - bboxes2[:, 1]

    area1 = w1 * h1
    area2 = w2 * h2
    center_x1 = (bboxes1[:, 2] + bboxes1[:, 0]) / 2
    center_y1 = (bboxes1[:, 3] + bboxes1[:, 1]) / 2
    center_x2 = (bboxes2[:, 2] + bboxes2[:, 0]) / 2
    center_y2 = (bboxes2[:, 3] + bboxes2[:, 1]) / 2

    inter_max_xy = torch.min(bboxes1[:, 2:],bboxes2[:, 2:])
    inter_min_xy = torch.max(bboxes1[:, :2],bboxes2[:, :2])
    out_max_xy = torch.max(bboxes1[:, 2:],bboxes2[:, 2:])
    out_min_xy = torch.min(bboxes1[:, :2],bboxes2[:, :2])

    inter = torch.clamp((inter_max_xy - inter_min_xy), min=0)
    inter_area = inter[:, 0] * inter[:, 1]
    inter_diag = (center_x2 - center_x1)**2 + (center_y2 - center_y1)**2
    outer = torch.clamp((out_max_xy - out_min_xy), min=0)
    outer_diag = (outer[:, 0] ** 2) + (outer[:, 1] ** 2)
    union = area1+area2-inter_area
    dious = inter_area / union - (inter_diag) / outer_diag
    dious = torch.clamp(dious,min=-1.0,max = 1.0)
    if exchange:
        dious = dious.T
    return dious

# 4.CIOU的提出

一个好的目标框回归损失应该考虑三个重要的几何因素：重叠面积、中心点距离、长宽比。
- GIoU：为了归一化坐标尺度，利用IoU，初步解决IoU为零的情况。
- DIoU：DIoU损失同时考虑了边界框的重叠面积和中心点距离。

anchor框和目标框之间的长宽比的一致性也是极其重要的

## 4.1.CIOU
$$
\begin{cases}
CIOU&=IOU-\frac{\rho(b,b^{gt})}{c^2}-\alpha \nu \\
\mathcal{L}_{CIOU}&=1-IOU+\frac{\rho(b,b^{gt})}{c^2} + \alpha \nu \\
\nu&=\frac{4}{\pi^2}(arctan\frac{\omega^{gt}}{h^{gt}}-arctan\frac{\omega}{h})^2 \\
\alpha&=\frac{\nu}{(1-IOU)+\nu}
\end{cases}
$$

CIOU比DIOU多出了$\alpha$和$\nu$这两个参数。其中$\alpha$是用于平衡比例的参数。$\nu$用来衡量anchor框和目标框之间的比例一致性。 从的定义式来看，损失函数会更加倾向于往重叠区域增多的方向优化，尤其是IOU为0的情况，这满足我们的要求。同时，在进行nms阶段，一般的评判标准是IOU，这个地方作者推荐替换为DIOU，这样考虑了中心点距离这一个信息，效果又有一定的提升。

In [ ]:
def bbox_overlaps_ciou(bboxes1, bboxes2):
    rows = bboxes1.shape[0]
    cols = bboxes2.shape[0]
    cious = torch.zeros((rows, cols))
    if rows * cols == 0:
        return cious
    exchange = False
    if bboxes1.shape[0] > bboxes2.shape[0]:
        bboxes1, bboxes2 = bboxes2, bboxes1
        cious = torch.zeros((cols, rows))
        exchange = True

    w1 = bboxes1[:, 2] - bboxes1[:, 0]
    h1 = bboxes1[:, 3] - bboxes1[:, 1]
    w2 = bboxes2[:, 2] - bboxes2[:, 0]
    h2 = bboxes2[:, 3] - bboxes2[:, 1]

    area1 = w1 * h1
    area2 = w2 * h2

    center_x1 = (bboxes1[:, 2] + bboxes1[:, 0]) / 2
    center_y1 = (bboxes1[:, 3] + bboxes1[:, 1]) / 2
    center_x2 = (bboxes2[:, 2] + bboxes2[:, 0]) / 2
    center_y2 = (bboxes2[:, 3] + bboxes2[:, 1]) / 2

    inter_max_xy = torch.min(bboxes1[:, 2:],bboxes2[:, 2:])
    inter_min_xy = torch.max(bboxes1[:, :2],bboxes2[:, :2])
    out_max_xy = torch.max(bboxes1[:, 2:],bboxes2[:, 2:])
    out_min_xy = torch.min(bboxes1[:, :2],bboxes2[:, :2])

    inter = torch.clamp((inter_max_xy - inter_min_xy), min=0)
    inter_area = inter[:, 0] * inter[:, 1]
    inter_diag = (center_x2 - center_x1)**2 + (center_y2 - center_y1)**2
    outer = torch.clamp((out_max_xy - out_min_xy), min=0)
    outer_diag = (outer[:, 0] ** 2) + (outer[:, 1] ** 2)
    union = area1+area2-inter_area
    u = (inter_diag) / outer_diag
    iou = inter_area / union
    with torch.no_grad():
        arctan = torch.atan(w2 / h2) - torch.atan(w1 / h1)
        v = (4 / (math.pi ** 2)) * torch.pow((torch.atan(w2 / h2) - torch.atan(w1 / h1)), 2)
        S = 1 - iou
        alpha = v / (S + v)
        w_temp = 2 * w1
    ar = (8 / (math.pi ** 2)) * arctan * ((w1 - w_temp) * h1)
    cious = iou - (u + alpha * ar)
    cious = torch.clamp(cious,min=-1.0,max = 1.0)
    if exchange:
        cious = cious.T
    return cious